BDLE 2023 TP2

révision: 19/09/2023

# TP2 2023 Fenetre Foursquare avec le système DuckDB (ETUDIANT)


# Préparation

In [1]:
import pandas as pd
from google.colab import data_table

def display(df, n=100):
  return data_table.DataTable(df.limit(n).df(), include_index=False, num_rows_per_page=10)

In [ ]:
!pip install duckdb==0.8.1

# Démarrer le service DuckDB

In [2]:
import duckdb

# db = duckdb.connect('foursquare')
db = duckdb.connect(':memory:')

# vérifier que le service fonctrionne
r = db.sql("SELECT 'hello' as col")
display(r)

,col
0,hello


# Données

In [3]:
import os
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

URL pour l'accès aux datasets

In [4]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4"
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


In [5]:
import os
from urllib import request
import zipfile

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

def unzip_file(local_dir, file):
  with zipfile.ZipFile(local_dir + "/" + file, 'r') as zip_ref:
    zip_ref.extractall(local_dir)
  # os.remove(local_dir + "/" + file)


web_dir = PUBLIC_DATASET + "/foursquare"

# ce fichier vient de kaggle : https://www.kaggle.com/datasets/chetanism/foursquare-nyc-and-tokyo-checkin-dataset
zip_filename = "dataset_TSMC2014_NYC.zip"


download_file(web_dir, local_dir, zip_filename)
unzip_file(local_dir, zip_filename)

# Liste des fichiers
os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/foursquare/dataset_TSMC2014_NYC.zip save in : /local/data/dataset_TSMC2014_NYC.zip


['dataset_TSMC2014_NYC.csv', 'dataset_TSMC2014_NYC.zip']

In [6]:
!head -4 /local/data/dataset_TSMC2014_NYC.csv

userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.71981038,-74.00258103,-240,Tue Apr 03 18:00:09 +0000 2012
979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.60679958,-74.04416981,-240,Tue Apr 03 18:00:25 +0000 2012
69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.71616168,-73.88307006,-240,Tue Apr 03 18:02:24 +0000 2012


# Table Visit

In [7]:
# duckdb.read_csv('/local/data/notes1M.json')

stmt = """
DROP TABLE if exists Visit;
CREATE TABLE Visit as
  SELECT *
  FROM '/local/data/dataset_TSMC2014_NYC.csv';
"""
db.sql(stmt)
print("done")

done


## Afficher le contenu d'une table

In [8]:
visit = db.table('Visit')
visit

┌────────┬──────────────────────┬──────────────────────┬───┬──────────────┬────────────────┬──────────────────────┐
│ userId │       venueId        │   venueCategoryId    │ … │  longitude   │ timezoneOffset │     utcTimestamp     │
│ int64  │       varchar        │       varchar        │   │    double    │     int64      │       varchar        │
├────────┼──────────────────────┼──────────────────────┼───┼──────────────┼────────────────┼──────────────────────┤
│    470 │ 49bbd6c0f964a520f4…  │ 4bf58dd8d48988d127…  │ … │ -74.00258103 │           -240 │ Tue Apr 03 18:00:0…  │
│    979 │ 4a43c0aef964a520c6…  │ 4bf58dd8d48988d1df…  │ … │ -74.04416981 │           -240 │ Tue Apr 03 18:00:2…  │
│     69 │ 4c5cc7b485a1e21e00…  │ 4bf58dd8d48988d103…  │ … │ -73.88307006 │           -240 │ Tue Apr 03 18:02:2…  │
│    395 │ 4bc7086715a7ef3bef…  │ 4bf58dd8d48988d104…  │ … │ -73.98251878 │           -240 │ Tue Apr 03 18:02:4…  │
│     87 │ 4cf2c5321d18a14395…  │ 4bf58dd8d48988d1cb…  │ … │ -73.9896583

In [9]:
display(visit,3)

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012


## Schéma d'une table

In [9]:
db.sql("""

SELECT column_name, data_type
FROM duckdb_columns
where table_name = 'Visit'

""")

┌─────────────────┬───────────┐
│   column_name   │ data_type │
│     varchar     │  varchar  │
├─────────────────┼───────────┤
│ userId          │ BIGINT    │
│ venueId         │ VARCHAR   │
│ venueCategoryId │ VARCHAR   │
│ venueCategory   │ VARCHAR   │
│ latitude        │ DOUBLE    │
│ longitude       │ DOUBLE    │
│ timezoneOffset  │ BIGINT    │
│ utcTimestamp    │ VARCHAR   │
└─────────────────┴───────────┘

# Requêtes

convertir le timestamp

In [10]:
visit2 = db.sql("""
  SELECT *, strptime(utcTimestamp, '%a %b %d %X %z %Y') as datetime
  FROM Visit
""")
visit2

┌────────┬──────────────────────┬───┬────────────────┬──────────────────────┬──────────────────────┐
│ userId │       venueId        │ … │ timezoneOffset │     utcTimestamp     │       datetime       │
│ int64  │       varchar        │   │     int64      │       varchar        │ timestamp with tim…  │
├────────┼──────────────────────┼───┼────────────────┼──────────────────────┼──────────────────────┤
│    470 │ 49bbd6c0f964a520f4…  │ … │           -240 │ Tue Apr 03 18:00:0…  │ 2012-04-03 18:00:0…  │
│    979 │ 4a43c0aef964a520c6…  │ … │           -240 │ Tue Apr 03 18:00:2…  │ 2012-04-03 18:00:2…  │
│     69 │ 4c5cc7b485a1e21e00…  │ … │           -240 │ Tue Apr 03 18:02:2…  │ 2012-04-03 18:02:2…  │
│    395 │ 4bc7086715a7ef3bef…  │ … │           -240 │ Tue Apr 03 18:02:4…  │ 2012-04-03 18:02:4…  │
│     87 │ 4cf2c5321d18a14395…  │ … │           -240 │ Tue Apr 03 18:03:0…  │ 2012-04-03 18:03:0…  │
│    484 │ 4b5b981bf964a52090…  │ … │           -240 │ Tue Apr 03 18:04:0…  │ 2012-04-03 18

In [11]:
visit3 = db.sql("""
  SELECT userId, venueId, datetime
  FROM visit2
""")

visit3

┌────────┬──────────────────────────┬──────────────────────────┐
│ userId │         venueId          │         datetime         │
│ int64  │         varchar          │ timestamp with time zone │
├────────┼──────────────────────────┼──────────────────────────┤
│    470 │ 49bbd6c0f964a520f4531fe3 │ 2012-04-03 18:00:09+00   │
│    979 │ 4a43c0aef964a520c6a61fe3 │ 2012-04-03 18:00:25+00   │
│     69 │ 4c5cc7b485a1e21e00d35711 │ 2012-04-03 18:02:24+00   │
│    395 │ 4bc7086715a7ef3bef9878da │ 2012-04-03 18:02:41+00   │
│     87 │ 4cf2c5321d18a143951b5cec │ 2012-04-03 18:03:00+00   │
│    484 │ 4b5b981bf964a520900929e3 │ 2012-04-03 18:04:00+00   │
│    642 │ 4ab966c3f964a5203c7f20e3 │ 2012-04-03 18:04:38+00   │
│    292 │ 4d0cc47f903d37041864bf55 │ 2012-04-03 18:04:42+00   │
│    428 │ 4ce1863bc4f6a35d8bd2db6c │ 2012-04-03 18:06:18+00   │
│    877 │ 4be319b321d5a59352311811 │ 2012-04-03 18:06:19+00   │
│     ·  │            ·             │           ·              │
│     ·  │            ·  

##  Ex1 Sequences de visites



### Numéro de POI

Définir les vues
 * venueId_poi(venueId, poi) qui associe le numéro original de venueId avec poi.
   * Indication, utiliser la fonction row_number() over( ...)
 * visit_poi(userId, poi, datetime) avec des numéros de poi allant de 1 à n



In [12]:
venueIds = db.sql("""
  select distinct venueId
  From visit3
""")

venueId_poi = db.sql("""
  select venueId,
    row_number() over (order by venueId) as poi
  From venueIds
""")
venueId_poi

┌──────────────────────────┬───────┐
│         venueId          │  poi  │
│         varchar          │ int64 │
├──────────────────────────┼───────┤
│ 3fd66200f964a52000e71ee3 │     1 │
│ 3fd66200f964a52000e81ee3 │     2 │
│ 3fd66200f964a52000f11ee3 │     3 │
│ 3fd66200f964a52001e51ee3 │     4 │
│ 3fd66200f964a52001e81ee3 │     5 │
│ 3fd66200f964a52002eb1ee3 │     6 │
│ 3fd66200f964a52003e51ee3 │     7 │
│ 3fd66200f964a52003e71ee3 │     8 │
│ 3fd66200f964a52003e81ee3 │     9 │
│ 3fd66200f964a52004e41ee3 │    10 │
│            ·             │     · │
│            ·             │     · │
│            ·             │     · │
│ 4b4f4d1bf964a5205a0027e3 │  9991 │
│ 4b4f593ef964a5207f0227e3 │  9992 │
│ 4b4f59def964a520a20227e3 │  9993 │
│ 4b4f5a31f964a520b20227e3 │  9994 │
│ 4b4f5dd8f964a520600327e3 │  9995 │
│ 4b4f601df964a520cf0327e3 │  9996 │
│ 4b4f6600f964a520cf0427e3 │  9997 │
│ 4b4f676cf964a520150527e3 │  9998 │
│ 4b4f676ff964a520160527e3 │  9999 │
│ 4b4f695ef964a520640527e3 │ 10000 │
├

In [13]:
visit_poi = db.sql("""
  Select v3.userId, vp.poi, v3.datetime
  From venueId_poi vp, visit3 v3
  Where vp.venueId = v3.venueId
""")

visit_poi

┌────────┬───────┬──────────────────────────┐
│ userId │  poi  │         datetime         │
│ int64  │ int64 │ timestamp with time zone │
├────────┼───────┼──────────────────────────┤
│    908 │     1 │ 2012-06-07 23:45:52+00   │
│    953 │     2 │ 2013-01-07 22:30:07+00   │
│    602 │     3 │ 2013-02-03 04:19:08+00   │
│    558 │     4 │ 2012-05-16 03:24:17+00   │
│    366 │     5 │ 2012-05-27 06:32:17+00   │
│    122 │     6 │ 2013-02-08 02:49:02+00   │
│    110 │     7 │ 2012-07-04 03:49:33+00   │
│    841 │     8 │ 2012-07-05 01:53:18+00   │
│    967 │     9 │ 2012-06-13 22:40:01+00   │
│     76 │    10 │ 2012-07-04 02:42:23+00   │
│      · │     · │           ·              │
│      · │     · │           ·              │
│      · │     · │           ·              │
│     14 │   951 │ 2012-06-04 23:13:46+00   │
│     10 │   952 │ 2012-05-11 01:27:17+00   │
│    442 │   955 │ 2012-06-02 22:09:14+00   │
│    164 │   958 │ 2013-01-26 01:48:51+00   │
│      1 │   963 │ 2012-04-09 16:2

### 1.1) Rang
Pour chaque utilisateur, ordonner les visites par date et leur attribuer un **rang** allant de 1 (plus ancienne) à n (la plus récente)

In [14]:
# verification
v_id199_p9602 = db.sql("""
  Select *
  From visit_poi
  Where userId = 199 and poi = 9602 and datetime = '2012-07-05 23:33:12+00'
""")
v_id199_p9602

┌────────┬───────┬──────────────────────────┐
│ userId │  poi  │         datetime         │
│ int64  │ int64 │ timestamp with time zone │
├────────┼───────┼──────────────────────────┤
│    199 │  9602 │ 2012-07-05 23:33:12+00   │
└────────┴───────┴──────────────────────────┘

In [15]:
visit_rank = db.sql("""
  select userId, poi, datetime,
    rank() over(partition by userId order by datetime) as rank
  From visit_poi
  order by userId, rank

""")
visit_rank

┌────────┬───────┬──────────────────────────┬───────┐
│ userId │  poi  │         datetime         │ rank  │
│ int64  │ int64 │ timestamp with time zone │ int64 │
├────────┼───────┼──────────────────────────┼───────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │
│      1 │   525 │ 2012-04-10 00:24:31+00   │     6 │
│      1 │  1241 │ 2012-04-10 03:36:56+00   │     7 │
│      1 │ 33237 │ 2012-04-10 16:21:48+00   │     8 │
│      1 │  2643 │ 2012-04-12 17:19:21+00   │     9 │
│      1 │ 33237 │ 2012-04-13 15:41:41+00   │    10 │
│      · │    ·  │           ·              │     · │
│      · │    ·  │           ·              │     · │
│      · │    ·  │           ·              │     · │
│     56 │  5511 │ 2012-12-22 01:22:30+00   │   308 │
│     56 │  3681 │ 2012-12-2

### 1.2) Date précédente et déplacement
 On veut connaitre la durée de déplacement séparant deux visites consécutives d'un meme utilisateur.
Pour chaque visite d'un utilisateur, ajouter l'attribut **prev_date** contenant la date de la visite qu'il a effectuée précédemment.

Puis ajouter l'attribut **interval_duration** calculé par différence entre la date courante et la date précédente.
Indication, voir la fonction *date_diff('sec', a, b)*

In [16]:
visit_prev_date = db.sql("""
  select userId, poi, datetime, rank,
    min(datetime) over (partition by userId order by rank rows between 1 preceding and 0 following) as prev_date
  From visit_rank
  order by userId, rank
""")
visit_prev_date

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 2012-04-08 20:02:10+00   │
│      1 │   525 │ 2012-04-10 00:24:31+00   │     6 │ 2012-04-09 16:20:52+00   │
│      1 │  1241 │ 2012-04-10 03:36:56+00   │     7 │ 2012-04-10 00:24:31+00   │
│      1 │ 33237 │ 2012-04-10 16:21:48+00   │     8 │ 2012-04-10 03:36:56+00   │
│      1 │  2643 │ 2012-04-1

In [17]:
visit_duration = db.sql("""
  select userId,poi,datetime,rank,prev_date, date_diff('sec',prev_date,datetime) as interval_duration
  from visit_prev_date
  order by userId, rank
""")

visit_duration

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┬───────────────────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │ interval_duration │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │       int64       │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┼───────────────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │                 0 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │            238253 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │             88685 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │              6101 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 2012-04-08 20:02:10+00   │             73122 │
│      1 │   525 │ 2012-04-10 00:24:31+00   │     6 │ 2012-04-09 16:20:52+00   │           

### 1.3) Numéro de trajectoire

Séparer les visites de chaque utilisateur en trajectoires.

  * Dans une trajectoire, la durée entre deux visites consécutives ne peut pas dépasser 1 jour (soit 24h * 3600 secondes).

Ajouter l'attribut **traj** indiquant le numéro de la trajectoire pour un utilisateur donné.
Indication: on peut utiliser une expression *case when then else end*

In [ ]:
24*3600

86400

In [18]:
visit_traj1 = db.sql("""
  select userId, poi, datetime, rank, prev_date, interval_duration,
    case
      when rank = 1 then 1
    else
      case when interval_duration > 24*3600 then 1
      else 0 end
    end as debut
  from visit_duration
  order by userId, rank
""")
visit_traj1

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┬───────────────────┬───────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │ interval_duration │ debut │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │       int64       │ int32 │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┼───────────────────┼───────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │                 0 │     1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │            238253 │     1 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │             88685 │     1 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │              6101 │     0 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 2012-04-08 20:02:10+00   │             73122 │     0 │
│      1 │   525 │ 

In [19]:
visit_traj2 = db.sql("""
  select userId, poi, datetime, rank, prev_date, interval_duration, debut,
    sum(debut) over (partition by userId order by rank) as traj
  from visit_traj1
  order by userId, rank

""")

visit_traj2

┌────────┬───────┬──────────────────────────┬───────┬──────────────────────────┬───────────────────┬───────┬────────┐
│ userId │  poi  │         datetime         │ rank  │        prev_date         │ interval_duration │ debut │  traj  │
│ int64  │ int64 │ timestamp with time zone │ int64 │ timestamp with time zone │       int64       │ int32 │ int128 │
├────────┼───────┼──────────────────────────┼───────┼──────────────────────────┼───────────────────┼───────┼────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │ 2012-04-04 23:31:31+00   │                 0 │     1 │      1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │ 2012-04-04 23:31:31+00   │            238253 │     1 │      2 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │ 2012-04-07 17:42:24+00   │             88685 │     1 │      3 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │ 2012-04-08 18:20:29+00   │              6101 │     0 │      3 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │     5 │ 20

## Ex2: Fenètres glissantes

### 2.1) Nombre de POI visités en une semaine

Ajouter l'attribut **nb_POI_7jours** donnant pour chaque jour où un utilisateur a fait des visites, le nombre de POI visités dans la semaine qui précède.

In [20]:
visit_nb_poi_7jours = db.sql("""
  select userId, poi, rank, debut, epoch(datetime) as date,
    count(*) over (partition by userId order by date range between 7*24*3600 preceding and 0 following) as nb_POI_7jours
  from visit_traj2
  order by userId, date

""")

visit_nb_poi_7jours

┌────────┬───────┬───────┬───────┬────────────┬───────────────┐
│ userId │  poi  │ rank  │ debut │    date    │ nb_POI_7jours │
│ int64  │ int64 │ int64 │ int32 │   int64    │     int64     │
├────────┼───────┼───────┼───────┼────────────┼───────────────┤
│      1 │  6428 │     1 │     1 │ 1333582291 │             1 │
│      1 │ 24554 │     2 │     1 │ 1333820544 │             2 │
│      1 │ 26196 │     3 │     1 │ 1333909229 │             3 │
│      1 │  4131 │     4 │     0 │ 1333915330 │             4 │
│      1 │   963 │     5 │     0 │ 1333988452 │             5 │
│      1 │   525 │     6 │     0 │ 1334017471 │             6 │
│      1 │  1241 │     7 │     0 │ 1334029016 │             7 │
│      1 │ 33237 │     8 │     0 │ 1334074908 │             8 │
│      1 │  2643 │     9 │     1 │ 1334251161 │             8 │
│      1 │ 33237 │    10 │     0 │ 1334331701 │             9 │
│      · │    ·  │     · │     · │      ·     │             · │
│      · │    ·  │     · │     · │      

### 2.2) Nombre cumulé de visites par utilisateur

Pour chaque utilisateur ajouter un attribut **cumul_visites** indiquant le nombre de visites qu'il a effectuées

In [21]:
# Il y a un plus qui se balade
# Il suffit d'une seconde requete pour le retirer
visit_cumul = db.sql("""
  select userId, poi, rank, debut, datetime,
    count(*) over (partition by userId order by datetime rows between unbounded preceding and current row) as cumul_visites
  from visit_traj2
  order by userId, datetime

""")
visit_cumul

┌────────┬───────┬───────┬───────┬──────────────────────────┬───────────────┐
│ userId │  poi  │ rank  │ debut │         datetime         │ cumul_visites │
│ int64  │ int64 │ int64 │ int32 │ timestamp with time zone │     int64     │
├────────┼───────┼───────┼───────┼──────────────────────────┼───────────────┤
│      1 │  6428 │     1 │     1 │ 2012-04-04 23:31:31+00   │             1 │
│      1 │ 24554 │     2 │     1 │ 2012-04-07 17:42:24+00   │             2 │
│      1 │ 26196 │     3 │     1 │ 2012-04-08 18:20:29+00   │             3 │
│      1 │  4131 │     4 │     0 │ 2012-04-08 20:02:10+00   │             4 │
│      1 │   963 │     5 │     0 │ 2012-04-09 16:20:52+00   │             5 │
│      1 │   525 │     6 │     0 │ 2012-04-10 00:24:31+00   │             6 │
│      1 │  1241 │     7 │     0 │ 2012-04-10 03:36:56+00   │             7 │
│      1 │ 33237 │     8 │     0 │ 2012-04-10 16:21:48+00   │             8 │
│      1 │  2643 │     9 │     1 │ 2012-04-12 17:19:21+00   │   

## Exercice 3

### 3.1) Visites dans une trajectoire
On constate qu'une trajectoire peut contenir plusieurs visites **consécutives** d'un même POI et on veut "fusionner" ces visites.
Pour chaque trajectoire, agréger les visites consécutives d'un meme POI. Définir les attributs date_début, date_fin pour la visite d'un POI.

Rmq1: s'il n'y a pas plusieurs visites consécutives pour un POI alors ses dates de début et de fin sont identiques.

Rmq2: une trajectoire peut contenir plusieurs visites non consécutives d'un même POI.

afficher les visites d'une trajectoire pour laqelle au moins une fusion a été faite.

In [37]:
# pas forcement consecutives
visit_consec = db.sql("""
  select userId, traj, debut, poi, datetime,
    min(datetime) over (partition by traj order by poi range between 0 preceding and 0 following) as min_date,
    max(datetime) over (partition by traj order by poi range between 0 preceding and 0 following) as max_date
  from visit_traj2
  order by userId, datetime

""")
visit_consec

┌────────┬────────┬───────┬───────┬──────────────────────────┬──────────────────────────┬──────────────────────────┐
│ userId │  traj  │ debut │  poi  │         datetime         │         min_date         │         max_date         │
│ int64  │ int128 │ int32 │ int64 │ timestamp with time zone │ timestamp with time zone │ timestamp with time zone │
├────────┼────────┼───────┼───────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│      1 │      1 │     1 │  6428 │ 2012-04-04 23:31:31+00   │ 2012-04-04 23:31:31+00   │ 2012-04-04 23:31:31+00   │
│      1 │      2 │     1 │ 24554 │ 2012-04-07 17:42:24+00   │ 2012-04-07 17:42:24+00   │ 2012-04-07 17:42:24+00   │
│      1 │      3 │     1 │ 26196 │ 2012-04-08 18:20:29+00   │ 2012-04-08 15:20:15+00   │ 2012-04-08 22:59:13+00   │
│      1 │      3 │     0 │  4131 │ 2012-04-08 20:02:10+00   │ 2012-04-08 20:02:10+00   │ 2012-04-08 20:02:10+00   │
│      1 │      3 │     0 │   963 │ 2012-04-09 16:20:52+00   │ 2

In [61]:
from typing import List, Tuple
def poi_consec(l: Tuple[int,int]) -> List[Tuple[int,int,int]]:
  l = sorted(l, key=lambda x : x[0])
  result = []
  i = 0
  while i<len(l):
    poi, date_debut, date_fin = l[i][1], l[i][0], l[i][0] # (poi, date_debut, date_fin)
    j = i+1
    while j<len(l) and l[j][1] == poi:
      date_fin = l[j][0]
      j += 1
    result.append((poi,date_debut,date_fin))
    i = j
  return r

db.create_function("poi_consec", poi_consec)

InvalidInputException: ignored

In [ ]:
visit_consec2 = db.sql("""
  select userId, traj, explode(poi_consec(collect_list((epoch(datetime),poi)))) as date_debut_fin
  from visit_traj2
  group by userId, traj
  order by userId

""")
visit_consec2